# Dataset Preprocessing

The following steps were taken during the dataset preprocessing:
- Columns "name" were renamed into "author_name" and "recipe_name" to eliminate any confusion. 
- During manual inspection, it was found that "author_note" column had hashtags and html tags in it. The html tags were removed using BeautifulSoup across all columns, along with the hashtags.
- "Steps" column contained unstructured text that included clearly separated steps. During the manual inspection, it was found that the steps are separated either with ",." or new lines. New lines in the dataset were respresented with:
    - *\n* or *\r\n*
    - *\n\n* or *\r\n\r\n*  
    
    Based on compilation environment. Newlines were replaced with *\n* to standardize the dataset.

    While single new lines appear to function as "sub steps" separators in some cases, it is still better to divide on them, to make steps more granular. Then, we evaluate whether each row of the dataset contains either of separators, to ensure that any other separators are not present. The output is were as follows: 
    ```
    Number of rows in column: 100
    Number of rows with '.,' separator: 100
    Number of rows with '\n\n' separator: 54
    Number of rows with '\n' separator: 59
    Number of rows with either ',.' or newlines: 100
    ```
    Concluding that other separators were not missed during manual inspection, the strings in the "steps" column were separated into lists. 
- Non-breaking spaces "\xa0" were removed from the dataset. While important in the Czech language, it is not relevant for the CLI application. 

# Data Indexing and Vector Store Creation
- Embedding model was chosen by filtering out open-source models that support multilinguality in [MTEB Leaderboard](https://huggingface.co/spaces/mteb/leaderboard). 
- The dataset from the csv file was imported into a list of documents using LangChain's `CSVLoader`, constructing one document per recipe/row. 
- To determine whether documents should be further splitted into text chunks, analysis of their lengths was conducted: 
    ```
    Maximum document length: 1344 characters
    Found in document index: 55
    =====================
    recipe_name: Kuře z jednoho hrnce
    author_name: Roman Vaněk
    ingredients: Česnek, Chilli koření mleté, Cuketa, Hřebíček mletý, Koriandr mletý, Krájená cibule, Kuřecí vykostěná stehna, Kuřecí vývar, Kurkuma mletá, Med, Mrkev, Olivový olej, Pepř černý , Plocholistá petržel, Pomerančová šťáva, Římský kmín mletý, Rýže, Skořice mletá, Sladká paprika, Sůl, Těstoviny, Víno vhodné k drůbežímu masu, Zázvor
    steps: ['Do kastrolu přidáme cuketu a mrkev, promícháme a pečeme 2 minuty. Občas zamícháme. Přidáme cibuli, promícháme a pečeme 1 minutu.', 'Kastrol vložíme do rozehřáté trouby a pečeme 40 minut. Před podáváním přimícháme nakrájenou petržel.', 'Ke koření přilijeme pomerančovou šťávu a vývar, přidáme med a pomerančovou kůru a promícháme. Přivedeme k varu a nalijeme do kastrolu ke kuřeti.', 'Maso vydatně osolíme a opepříme z obou stran.', 'Mrkev nakrájíme na kolečka silná 0,5 cm. Cibuli nakrájíme najemno. Česnek nakrájíme na plátky. Cuketu rozkrojíme podélně na čtvrtiny a ty pak nakrájíme na plátky silné 0,5 cm. Zázvor nastrouháme a šťávu z něj vymačkáme na mrkev.', 'Přidáme česnek a zázvor, promícháme a za občasného míchání pečeme 2 minuty. Kastrol stáhneme z ohně a vyskládáme do něj opečené maso kůží nahoru.', 'Troubu rozehřejeme na 180 °C.', 'V kastrolu rozpálíme na vyšším ohni olej a maso na něm opečeme 4 minuty dozlatova na s.']
    ```

    
    1344 characters is far from any embedding model's or LLM's context limit, so no further splitting was performed.
- **ChromaDB** was chosen as a database consideting the previous experience with it. However, more production-friednly option would be FAISS usage, due to it's speed and retrieval accuracy.
- Fields of CSV such as "id" or "authors_note" were added to the vector store as metadata, not being indexed themselves. While it is possible that in the future dataset iterations "author_note" will contain informative recipe-related data, it is not the case now, as it purely contains promotional information. 

# Retrieval Augmented Generation
- **Langchain** was chosen as a library to create a RAG chain. 
- OpenAIs **GPT-4o** is used as a language model. However, this choice is made due to the convenience of implementation, so one of the top future priorities is to replace it with a more cost-effective, fast, intelligent model (e.g., Gemini Flash 2.0).
- The user can interact with the chatbot by running **main.py** and sending/recieving messages through the CLI. 
- A method for **context display** was implemented. Whether user want's to display retrieved context along with the models response, they should type "provide_context" to turn on this functionality. The context will be displayed in a nicely formatted way.

# RAG Evaluation
*Note: This is an experimental section which is still under research.*

For RAG evaluation, I applied LLM-as-a-judge approach using DeepEvals metrics. 
- Firstly, QA pairs pairs were generated using Gemini Flash 2.0. 
- Secondly, the QA pairs were filtered using critique agents, based on the following criteria: groundness, relevance and how standalone they are, inspired by the the [HuggingFace Cookbook RAG Evaluation Example](https://huggingface.co/learn/cookbook/en/rag_evaluation)
- Thirdly, the QA pairs were evaluated using DeepEvals metrics such as "Answer Relevancy" along with Gemini Flash 2.0. 

However, I am concluding that the evaluation approach was not perfectly aligned with the task, as the main purpose of the RAG chatbot is guide users through. I believe that only in rare cases will users ask specific questions about concrete recipes, if the full recipe was already outputted by the chatbot.

Therefore, further exploration of evaluation methods is needed. 

# Future Considerations

- One of the main improvements to perform is exploration of the speed and cost optimization for the chatbot. This involves:
    - Making the chatbot more responsive by streaming the output instead of outputting the whole message in one go. 
    - Changing the model to a more efficient one. This can be determined, for example, using [ArtificialAnalysis](https://artificialanalysis.ai/) boards.
- Advanced RAG improvements such as retrieved context reranking. 
- Possible implementation of **agentic system**, which will determine on the run whether to output the full formatted recipe to the user after each user message. While this might not have an obvious advantages in the CLI bot, in the real-life application the chatbot might generate recipte cards from the retrieved context, which will also include the link to the discussion, etc. 